In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd /content/gdrive/MyDrive/training/

# Colab warns and provides remediation steps if the GPUs is not compatible with RAPIDS.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

In [1]:
import numpy as np
import pandas as pd
import cudf
import cupy
import dask_cudf
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error
import rapids_qt
from dask_cuda import LocalCUDACluster
from dask.distributed import Client
import gc

cluster = LocalCUDACluster()
client = Client(cluster)

/home/aikawa/anaconda3/envs/rapids-24.10/lib/python3.12/site-packages/distributed/node.py:187: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 46189 instead
  warnings.warn(


In [2]:
V = cudf.read_csv('data/Liver_V.csv', header=0, index_col=0)
V.index.name = None

# ==== rankの値を自由に変更して実行 ====
rank = 10
# ====================================

model = NMF(n_components=rank, init='nndsvd', random_state=42)
V = V.to_pandas()
W = model.fit_transform(V)
H = model.components_

/home/aikawa/anaconda3/envs/rapids-24.10/lib/python3.12/site-packages/sklearn/decomposition/_nmf.py:1741: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [3]:
df = dask_cudf.read_csv('data/sample.csv')
df = df.persist()

df['global_x'] = df['global_x'] - df['global_x'].min().compute()
df['global_y'] = df['global_y'] - df['global_y'].min().compute()

df['global_x'] = df['global_x'].astype('int')
df['global_y'] = df['global_y'].astype('int')

top = 0
bottom = df['global_y'].max().compute()
left = 0
right = df['global_x'].max().compute()

In [4]:
genes_list = df['gene'].unique().compute().to_pandas().tolist()
filtered_genes = [i for i in genes_list if 'Blank' not in i]
gene_to_channel = {gene: i for i, gene in enumerate(filtered_genes)}

cp_matrix = cupy.zeros((bottom - top + 1, right - left + 1, len(filtered_genes)), dtype=np.int32)

for _, row in df.compute().to_pandas().iterrows():
    x = row['global_x']
    y = row['global_y']
    gene = row['gene']
    if gene in filtered_genes:
        channel = gene_to_channel[gene]
        cp_matrix[y, x, channel] += 1
    else:
        continue
        
W = cudf.DataFrame(W, index=V.index)
quad_tree_generator = rapids_qt.QuadTreeGenerator(arr=cp_matrix, gene_to_channnel=gene_to_channel, rank=rank, pattern=W, iterations=341, min_size=20)
quad_tree_generator.generate_quad_tree()
quad_tree_generator.save_gif(output_path=f"output/sample.gif")
quad_tree_generator.save_png(output_path=f"output/sample.png")
del cp_matrix
del quad_tree_generator.arr
gc.collect()
print('done.')


done.
